# Packages 

In [31]:
%load_ext autoreload
%autoreload 2
import sys
import logging
import pandas as pd
base_dir = '../'
sys.path.append(base_dir)
import os
import numpy as np
import polars as pl
from src.config import raw_data_dir, raw_data_session_id_dir# task1_train_eval_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

# Download data

In [7]:
!mkdir ../{raw_data_dir}
!mkdir ../{raw_data_session_id_dir}
!mkdir ../{task1_train_eval_dir}

mkdir: cannot create directory ‘../data/raw_data_session_id’: File exists


In [3]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/4oimPj_fUBFa71BqyptNVbbd0RkHbTBC28WZ6jAIdB4
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/4oimPj_fUBFa71BqyptNVbbd0RkHbTBC28WZ6jAIdB4'
API Key valid
Gitlab access token valid
Saved details successfully!


In [10]:
!aicrowd dataset list --challenge task-1-next-product-recommendation 
# ../{raw_data_dir}

                          Datasets for challenge #1116                          
┌───┬──────────────────────────────┬───────────────────────────────┬───────────┐
│ # │ Title                        │ Description                   │      Size │
├───┼──────────────────────────────┼───────────────────────────────┼───────────┤
│ 0 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │  19.37 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task1_phase1… │                               │           │
│ 1 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │   1.92 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task2_phase1… │                               │           │
│ 2 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │   2.67 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task3_p

In [11]:
!aicrowd dataset download --challenge task-1-next-product-recommendation

sessions_test_task2_phase1.csv:   0%|               | 0.00/1.92M [00:00<?, ?B/s]
sessions_test_task1.csv:   0%|                      | 0.00/19.3M [00:00<?, ?B/s]

sessions_test_task2_phase1.csv: 100%|██████| 1.92M/1.92M [00:00<00:00, 13.0MB/s]

sessions_test_task1.csv:   5%|▋            | 1.05M/19.3M [00:00<00:02, 6.64MB/s]

sessions_test_task3.csv: 100%|█████████████| 2.67M/2.67M [00:00<00:00, 10.6MB/s]
sessions_test_task2.csv:   0%|                      | 0.00/1.91M [00:00<?, ?B/s]
sessions_test_task1.csv:  43%|█████▋       | 8.39M/19.3M [00:00<00:00, 14.6MB/s]

sessions_test_task2.csv:  55%|███████      | 1.05M/1.91M [00:00<00:00, 6.65MB/s]
sessions_test_task2.csv: 100%|█████████████| 1.91M/1.91M [00:00<00:00, 8.85MB/s]
sessions_test_task1_phase1.csv:   0%|               | 0.00/19.4M [00:00<?, ?B/s]


products_train.csv:   0%|                            | 0.00/589M [00:00<?, ?B/s]

sessions_test_task3_phase1.csv: 100%|██████| 2.67M/2.67M [00:00<00:00, 11.9MB/s]
sessions_test_task1_p

In [13]:
raw_data_dir

'data/raw_data'

In [12]:
! ls {raw_data_dir}

ls: cannot access 'data/raw_data': No such file or directory


# Preprocess data 

## Test data from phase 1 and 2 

In [14]:
task1_phase1 = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task1_phase1.csv'))
task1_phase2 = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task1.csv'))

In [17]:
print(task1_phase1.collect().shape)
print(task1_phase2.collect().shape)

(316971, 2)
(316972, 2)


In [18]:
print(task1_phase1.collect().head())
print(task1_phase2.collect().head())

shape: (5, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B08V12CT4C' 'B08V1KXBQD' 'B01B… ┆ DE     │
│ ['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R… ┆ DE     │
│ ['B07YSRXJD3' 'B07G7Q5N6G' 'B08C… ┆ DE     │
│ ['B08KQBYV43' '3955350843' '3955… ┆ DE     │
│ ['B09FPTCWMC' 'B09FPTQP68' 'B08H… ┆ DE     │
└───────────────────────────────────┴────────┘
shape: (5, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B087VLP2RT' 'B09BRQSHYH' 'B099… ┆ DE     │
│ ['B08XW4W667' 'B096VMCJYF' 'B096… ┆ DE     │
│ ['B09Z4T2GJ3' 'B09Z3FBXMB' 'B093… ┆ DE     │
│ ['B07T6Y2HG7' 'B07T2NBLX9' 'B07Y… ┆ DE     │
│ ['B0B2DRKZ6X' 'B0B2DRKZ6X' 'B0

In [21]:
print(task1_phase1.select('locale').collect().to_series().value_counts())
print(task1_phase2.select('locale').collect().to_series().value_counts())

shape: (3, 2)
┌────────┬────────┐
│ locale ┆ counts │
│ ---    ┆ ---    │
│ str    ┆ u32    │
╞════════╪════════╡
│ JP     ┆ 96467  │
│ UK     ┆ 115936 │
│ DE     ┆ 104568 │
└────────┴────────┘
shape: (3, 2)
┌────────┬────────┐
│ locale ┆ counts │
│ ---    ┆ ---    │
│ str    ┆ u32    │
╞════════╪════════╡
│ UK     ┆ 115937 │
│ JP     ┆ 96467  │
│ DE     ┆ 104568 │
└────────┴────────┘


In [22]:
task3_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task3.csv'))

In [25]:
# task3_pl.head().collect()

# adding session_id to all data

In [2]:
train_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_train.csv'))
task1_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task1.csv'))
task2_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task2.csv'))
task3_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task3.csv'))
product_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'products_train.csv'))

In [3]:
length_lst = []
for df in [train_pl, task1_pl, task2_pl, task3_pl, product_pl]:
    print()
    print()
    print(df.schema)
    shape = df.collect().shape
    length_lst.append(shape[0])
    print(shape)
    print()
    print(df.head(3).collect())
    print()
    print()



{'prev_items': Utf8, 'next_item': Utf8, 'locale': Utf8}
(3606249, 3)

shape: (3, 3)
┌───────────────────────────────────┬────────────┬────────┐
│ prev_items                        ┆ next_item  ┆ locale │
│ ---                               ┆ ---        ┆ ---    │
│ str                               ┆ str        ┆ str    │
╞═══════════════════════════════════╪════════════╪════════╡
│ ['B09W9FND7K' 'B09JSPLN1M']       ┆ B09M7GY217 ┆ DE     │
│ ['B076THCGSG' 'B007MO8IME' 'B08M… ┆ B001B4THSA ┆ DE     │
│ ['B0B1LGXWDS' 'B00AZYORS2' 'B0B1… ┆ B0767DTG2Q ┆ DE     │
└───────────────────────────────────┴────────────┴────────┘




{'prev_items': Utf8, 'locale': Utf8}
(316971, 2)

shape: (3, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B08V12CT4C' 'B08V1KXBQD' 'B01B… ┆ DE     │
│ ['B00R9R5ND6' 'B00R9RZ

In [4]:
length_lst

[3606249, 316971, 34691, 56423, 1551057]

In [15]:
train_pl = train_pl.with_columns(
    pl.Series(name='session_id', values=range(length_lst[0]))
    , pl.Series(name='random', values=np.random.uniform(size=length_lst[0]))
)
task1_pl= task1_pl.with_columns(
    pl.Series(name='session_id', values=range(length_lst[0],sum(length_lst[:2])))
)
task2_pl= task2_pl.with_columns(
    pl.Series(name='session_id', values=range(sum(length_lst[:2]),sum(length_lst[:3])))
)
task3_pl= task3_pl.with_columns(
    pl.Series(name='session_id', values=range(sum(length_lst[:3]),sum(length_lst[:4])))
)

In [16]:
# assert train_pl.select('session_id').max().collect()<200

In [17]:
train_pl.select('session_id').max().collect()

session_id
i64
3606248


In [53]:
task1_pl.select('session_id').min().collect()

session_id
i64
3606249


In [54]:
task2_pl.select('session_id').min().collect()

session_id
i64
3923220


In [55]:
task3_pl.select('session_id').min().collect()

session_id
i64
3957911


## Save data with session_id 

In [18]:
eval_df = train_pl.filter(pl.col('random')<=0.1).select(pl.all().exclude('random'))
train_df = train_pl.filter(pl.col('random')>0.1).select(pl.all().exclude('random'))

In [19]:
eval_df.collect().shape

(359776, 4)

In [20]:
train_df.collect().shape

(3246473, 4)

In [21]:
train_pl.collect().shape

(3606249, 5)

In [22]:
train_df.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))
eval_df.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'))

In [66]:

task1_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'))
task2_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2.parquet'))
task3_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'))

## Production data 

In [82]:
value_counts = product_pl.with_columns(pl.count().over('id').alias('cnt')).select(['id', 'cnt']).unique().collect().to_pandas()['cnt'].value_counts()

In [84]:
value_counts

cnt
1    1311106
2      72977
3      16368
4       6738
5       2975
6        511
Name: count, dtype: int64

In [86]:
value_counts/value_counts.sum()

cnt
1    0.929417
2    0.051732
3    0.011603
4    0.004776
5    0.002109
6    0.000362
Name: count, dtype: float64

**92% of products only in a single country; the rests are in more than one country**

### A example 

In [72]:
product_pl.with_columns(pl.count().over('id').alias('cnt')).sort(['cnt'], descending=True).head().collect()

id,locale,title,price,brand,color,size,model,material,author,desc,cnt
str,str,str,f64,str,str,str,str,str,str,str,u32
"""B0B9W7W8JD""","""FR""","""RAVIAD Câble U…",6.99,"""RAVIAD""","""Argent""","""0.3M""","""H011-SI-030""",null,null,"""【Compatibilité…",6
"""B09YMV3TXX""","""DE""","""Spigen Ultra H…",16.99,"""Spigen""","""Crystal Clear""",null,"""ACS04726""","""Polycarbonat""",null,"""Die Konstrukti…",6
"""B005EJH6RW""","""DE""","""Amazon Basics …",9.69,"""Amazon Basics""","""Schwarz""","""1-Pack""","""MSU0939""",null,null,"""High-definitio…",6
"""B07ZF8WFZN""","""IT""","""Cavo da HDMI a…",11.99,"""BENFEI""","""1 Confezioni""","""0,9 metri""","""000282black""",null,null,"""18 MESI DI GAR…",6
"""B075GQ87PV""","""JP""","""レゴ(LEGO) クラシック…",2345.0,"""レゴ(LEGO)""",null,null,"""10713""",null,null,"""かしこく収納""",6


In [74]:
product_pl.filter(pl.col('id')=='B0B9W7W8JD').collect()

id,locale,title,price,brand,color,size,model,material,author,desc
str,str,str,f64,str,str,str,str,str,str,str
"""B0B9W7W8JD""","""DE""","""USB C Kabel Ku…",6.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【Beidseitig nu…"
"""B0B9W7W8JD""","""JP""","""RAVIAD USB Typ…",699.0,"""RAVIAD""","""シルバー""","""0.3M""","""H011-SI-030""",null,null,"""【卓越した耐久性】高耐久を持…"
"""B0B9W7W8JD""","""UK""","""Short USB C Ca…",5.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【24-Month Warr…"
"""B0B9W7W8JD""","""ES""","""RAVIAD Cable U…",6.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【Carga Rápida …"
"""B0B9W7W8JD""","""FR""","""RAVIAD Câble U…",6.99,"""RAVIAD""","""Argent""","""0.3M""","""H011-SI-030""",null,null,"""【Compatibilité…"
"""B0B9W7W8JD""","""IT""","""RAVIAD Cavo US…",6.99,"""RAVIAD""","""Argento""","""0.3M""","""H011-SI-030""",null,null,"""【Connettore Re…"


### Save product info 

In [87]:
product_pl = product_pl.with_columns(
    pl.Series(name='unique_id', values=range(length_lst[-1]))
)

In [88]:
product_pl.schema

{'id': Utf8,
 'locale': Utf8,
 'title': Utf8,
 'price': Float64,
 'brand': Utf8,
 'color': Utf8,
 'size': Utf8,
 'model': Utf8,
 'material': Utf8,
 'author': Utf8,
 'desc': Utf8,
 'unique_id': Int64}

In [89]:
product_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'products_train.parquet'))


In [ ]:
product_df = product_pl.select(['unique_id', 'id']).collect().to_pandas()
product_unique2id = dict(zip(product_df['unique_id'].values, product_df['id'].values))
with open(os.path.join(base_dir, raw_data_session_id_dir, 'product_unique2id.json'), 'wb') as f:
    pickle.dump(product_unique2id, f)

# assign session_id to test data of phase2

In [4]:
raw_data_session_id_dir

'data/raw_data_session_id'

In [13]:
train_df = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))
eval_df = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'))
task1_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1_phase1.parquet'))
task2_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2_phase1.parquet'))
task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3_phase1.parquet'))

In [14]:
length_lst = []
min_max_id = []
for df in [train_df, eval_df, task1_pl, task2_pl, task3_pl,# product_pl
          ]:
    # print()
    # print()
    # print(df.schema)
    min_id = df.select('session_id').min().collect()
    max_id = df.select('session_id').max().collect()
    shape = df.collect().shape
    length_lst.append(shape[0])
    min_max_id.append([min_id, max_id])
    # print(shape)
    # print()
    # print(df.head(3).collect())
    # print()
    # print()

In [15]:
length_lst

[3246473, 359776, 316971, 34691, 56423]

In [16]:
# min_max_id

In [17]:
max_id = 4014333


In [20]:
assert sum(length_lst) == max_id+1

In [36]:
task1_phase2 = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task1.csv'))
task2_phase2 = pl.from_pandas(pd.read_csv(os.path.join('..', raw_data_dir, 'sessions_test_task2.csv'))).lazy()
task3_phase2 = pl.from_pandas(pd.read_csv(os.path.join('..', raw_data_dir, 'sessions_test_task3.csv'))).lazy()
# product_ = pl.scan_csv(os.path.join('..', raw_data_dir, 'products_train.csv'))

In [37]:
assert task1_phase2.filter(pl.col('locale').is_null()).collect().shape[0] == 0

In [38]:
assert task2_phase2.filter(pl.col('locale').is_null()).collect().shape[0] == 0

In [39]:
assert task3_phase2.filter(pl.col('locale').is_null()).collect().shape[0] == 0

In [40]:
phase2_length_lst = []
for df in [task1_phase2, task2_phase2, task3_phase2]:
    print()
    print()
    print(df.schema)
    shape = df.collect().shape
    phase2_length_lst.append(shape[0])
    print(shape)
    print()
    print(df.head(3).collect())
    print()
    print()



{'prev_items': Utf8, 'locale': Utf8}
(316972, 2)

shape: (3, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B087VLP2RT' 'B09BRQSHYH' 'B099… ┆ DE     │
│ ['B08XW4W667' 'B096VMCJYF' 'B096… ┆ DE     │
│ ['B09Z4T2GJ3' 'B09Z3FBXMB' 'B093… ┆ DE     │
└───────────────────────────────────┴────────┘




{'prev_items': Utf8, 'locale': Utf8}
(34690, 2)

shape: (3, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B07GTS7SWK' 'B07GTS7SWK']       ┆ ES     │
│ ['B0B33YWVHR' '849988993X' 'B09Q… ┆ ES     │
│ ['B08FMPXDTJ' 'B0B4612MTM' 'B0B4… ┆ ES     │
└───────────────────────────────────┴────────┘




{'prev_items': Ut

In [41]:
max_id

4014333

In [42]:
phase2_length_lst

[316972, 34690, 56422]

In [43]:
# phase2_l?ength_lst[:2]

In [44]:
task1_phase2= task1_phase2.with_columns(
    pl.Series(name='session_id', values=range(max_id+1,sum(phase2_length_lst[:1])+max_id+1))
)
task2_phase2= task2_phase2.with_columns(
    pl.Series(name='session_id', values=range(sum(phase2_length_lst[:1])+max_id+1,sum(phase2_length_lst[:2])+max_id+1))
)
task3_phase2= task3_phase2.with_columns(
    pl.Series(name='session_id', values=range(sum(phase2_length_lst[:2])+max_id+1,sum(phase2_length_lst[:3])+max_id+1))
)

In [45]:
# length_lst = []
# min_max_id = []
# for df in [train_df, eval_df, task1_pl, task2_pl, task3_pl,# product_pl
#           ]:
#     # print()
#     # print()
#     # print(df.schema)
#     min_id = df.select('session_id').min().collect()
#     max_id = df.select('session_id').max().collect()

    
# phase2_length_lst = []
for df in [task1_phase2, task2_phase2, task3_phase2]:
    min_id = df.select('session_id').min().collect()
    max_id = df.select('session_id').max().collect()
    print(f"[{min_id}, {max_id}]")
    # print()
    # print()
    # print(df.schema)
    # shape = df.collect().shape
    # phase2_length_lst.append(shape[0])
    # print(shape)
    # print()
    # print(df.head(3).collect())
    # print()
    # print()

[shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4014334    │
└────────────┘, shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4331305    │
└────────────┘]
[shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4331306    │
└────────────┘, shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4365995    │
└────────────┘]
[shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4365996    │
└────────────┘, shape: (1, 1)
┌────────────┐
│ session_id │
│ ---        │
│ i64        │
╞════════════╡
│ 4422417    │
└────────────┘]


In [46]:

task1_phase2.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'))
task2_phase2.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2.parquet'))
task3_phase2.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'))

# Split train data to 2 parts 

In [13]:
train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))


In [14]:
train_pl.head().collect()

prev_items,next_item,locale,session_id
str,str,str,i64
"""['B09W9FND7K' …","""B09M7GY217""","""DE""",0
"""['B076THCGSG' …","""B001B4THSA""","""DE""",1
"""['B0B1LGXWDS' …","""B0767DTG2Q""","""DE""",2
"""['B0749V8TC7' …","""B0749TX4YS""","""DE""",5
"""['B09SMK3R8H' …","""B08YNZT93Z""","""DE""",6


In [15]:
length = train_pl.collect().shape[0]
length

3246473

In [28]:
train_pl.select('locale').collect().to_series().value_counts()

locale,counts
str,u32
"""FR""",105940
"""ES""",80083
"""JP""",881273
"""DE""",1000634
"""IT""",114177
"""UK""",1064366


In [16]:
length 

3246473

In [17]:
train_pl = train_pl.with_columns(
    pl.Series(name='random', values=np.random.uniform(size=length))
)

In [18]:
train1 = train_pl.filter(pl.col('random')<=0.5).select(pl.all().exclude('random'))
train2 = train_pl.filter(pl.col('random')>0.5).select(pl.all().exclude('random'))

In [23]:
assert train1.collect().shape[0] + train2.collect().shape[0] == length

In [24]:
train1.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train1.parquet'))


In [25]:
train2.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train2.parquet'))
